In [ ]:
%reload_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import numpy as np
from pyomo.opt import SolverFactory
from tqdm.notebook import tqdm

from pyomo_models.task_allocation import TaskAllocationModel, Scenario, Parameters

In [ ]:
sc = Scenario.from_file("scenarios/task_allocation/students.yaml")

In [ ]:
N_t = 10
v = np.array([1, 1])
r_charge = np.array([0.02, 0.02, 0.02])
r_deplete_fly = np.array([0.00125, 0.00125, 0.00125])
r_deplete_inspect = np.array([0.01, 0.01, 0.01])
i = np.array([1, 1, 2])
B_start = [1, 1]
B_min = 0
B_max = 1
R = np.array([30, 30, 30, 30, 30, 30, 60, 60, 60])
parameters = {p: eval(p) for p in ['N_d', 'N_t', 'v', 'r_charge', 'r_deplete_fly', 'r_deplete_inspect', 'i', 'B_start', 'B_min', 'B_max', 'R']}

In [ ]:
model = TaskAllocationModel(sc, parameters)

In [ ]:
solution = SolverFactory('gurobi').solve(model, tee=True, timelimit=300) 

In [ ]:
print(model.execution_time())

In [ ]:
N_t_candidates = range(2, 9)
execution_times = {}
for N_t in tqdm(N_t_candidates):
    parameters = {p: eval(p) for p in ['N_d', 'N_t', 'v', 'r_charge', 'r_deplete', 'i', 'B_start', 'B_min', 'B_max', 'R']}
    model = TaskAllocationModel(sc, parameters)
    solution = SolverFactory('gurobi').solve(model) 
    execution_times[N_t] = model.execution_time()

In [ ]:
print(execution_times)

In [ ]:
np.reshape(model.P[:,:,:](), (model.N_d, model.N_n, model.N_t)).round(2)

In [ ]:
np.reshape(model.I[:,:](), (model.N_d, model.N_t)).round(2)

In [ ]:
np.reshape(model.b[:,:](), (model.N_d, model.N_t)).round(2)

In [ ]:
for d in model.d:
    print(model.inspection_time(d)())

In [ ]:
for d in model.d:
    print(model.moving_time(d)())

In [ ]:
for d in model.d: 
    print(model.charging_time(d)())

In [ ]:
np.reshape(model.b_min[:,:](), (model.N_d, model.N_t-1)).round(2)

In [ ]:
np.reshape(model.b_plus[:,:](), (model.N_d, model.N_t-1)).round(2)